# Flexion Angle Detection

The following cells are able to plot a human pose estimate on a pre-recorded video stream and get the angle between specific joints of the right arm.

## Key Aspects
- Runs on OpenCV - machine learning applicable library for computer vision
- Real time application
- Can run on CPU (later to be optimized for Nvidia GPU cores)



In [1]:
#importing required packages and dependancies
import cv2
import numpy as np
import matplotlib as plt
import math
import time
import mediapipe as mp
import pandas as pd
import datetime
import csv

## Defining a class poseDetector with predefined joints pose, position and angle functions
The cell below is to initiate the pose detector class as directed from MediaPipe with various parameters in the class for pose detection.

We can have different classes for face, hand, pose and the angle but all of them have associated attributes hence a single class called poseDetector with diferent functions to be called later on in the notebook.

In [6]:
class poseDetector():

    def __init__(self, mode=False, complexity=1, smooth_landmarks=True, 
                 segmentation=False, smooth_segmentation=True, detectionCon=0.5, trackCon=0.5):
        
        self.mode = mode
        self.complexity = complexity
        self.smooth_landmarks = smooth_landmarks
        self.segmentation = segmentation
        self.smooth_segmentation = smooth_segmentation
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpDraw = mp.solutions.drawing_utils
        self.mpPose = mp.solutions.pose
        self.pose = self.mpPose.Pose(self.mode, 
                                     self.complexity, 
                                     self.smooth_landmarks,
                                     self.segmentation,
                                     self.smooth_segmentation,
                                     self.detectionCon, 
                                     self.trackCon)
        
    #function to find the pose    
    def findPose(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)
        if self.results.pose_landmarks:
            if draw:
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks,
                                           self.mpPose.POSE_CONNECTIONS)
        return img
    
    
    #funtion to return the position of all the joints in the image/video
    def findPosition(self, img, draw=True):
        self.lmList = []
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
                # print(id, lm)
                cx, cy = int(lm.x * w), int(lm.y * h)
                # Getting the current date and time
                ts = time.time()
                # dt = datetime.now()

                # getting the timestamp
                # ts = datetime.timestamp(dt)
                
                self.lmList.append([id, cx, cy, ts])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
        return self.lmList
    
    
    #function to return the angle between joints p1,p2,p3
    def findAngle(self, img, p1, p2, p3, draw=True):
        # Get the landmarks/positions of the joints
        x1, y1 = self.lmList[p1][1:3]
        x2, y2 = self.lmList[p2][1:3]
        x3, y3 = self.lmList[p3][1:3]

        # Calculate the Angle between the mapped joints
        angle = math.degrees(math.atan2(y3 - y2, x3 - x2) -
                             math.atan2(y1 - y2, x1 - x2))
        if angle < 0:
            angle += 360
        if angle > 180:
            angle = 360-angle
        # print(angle)

        # Draw the selected joints and the line joining them
        if draw:
            #highlighting the skeleton connecting the joints
            cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 3)
            cv2.line(img, (x3, y3), (x2, y2), (255, 255, 255), 3)
            #highlighting the joints with bigger circles
            cv2.circle(img, (x1, y1), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x1, y1), 15, (0, 0, 255), 2)
            cv2.circle(img, (x2, y2), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 15, (0, 0, 255), 2)
            cv2.circle(img, (x3, y3), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x3, y3), 15, (0, 0, 255), 2)
            # Drawing the value of the angle on the highlighted joint
            cv2.putText(img, str(int(angle)), (x2 - 50, y2 + 50),
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        return angle

## Pose on image from poseDetector
The cell below is capable of plotting the human pose on images/videos of human beings in different postures.

The poseDetector class defined in the cells above is imported and a defined function is used to plot the human pose on the image/video provided.

In [10]:
cap = cv2.VideoCapture("TestVideo/FlexTest.mp4")
detector = poseDetector() #from class poseDetector in cells above 

while True:
    #for video input
    #success, img = cap.read()
    #for image input
    img = cv2.imread("TestVideo/test1.jpg")
    
    img = cv2.resize(img, (900,700))
    #img = cv2.flip(img, 0)

    img = detector.findPose(img, False)
    lmList = detector.findPosition(img, False)
    print(lmList)
    
    if len(lmList) != 0:
        file = open("C:/Users/dsail/Desktop/Projects/flexion-angle/TestVideo/output.txt", "w")
        csv.writer(file, delimiter=',').writerows(lmList)
        file.close()
        
    #with open("C:/Users/dsail/Desktop/Projects/flexion-angle/TestVideo/output.txt", "w") as f:
        #csv.writer(f, delimiter=',').writerows(lmList)
           
    
    #Defining the 3 joints whose angle you need to find
    if len(lmList) != 0:
        #left leg
        detector.findAngle(img, 23, 25, 27)
        #right leg
        #detector.findAngle(img, 24, 26, 28)
        #right arm
        #detector.findAngle(img, 12, 14, 16)
        #left arm
        detector.findAngle(img, 11, 13, 15)
        
                
    cv2.imshow("image", img)
    cv2.waitKey(1)
    



[[0, 778, 463, 1663918149.437315], [1, 792, 454, 1663918149.437315], [2, 792, 453, 1663918149.437315], [3, 792, 452, 1663918149.437315], [4, 792, 451, 1663918149.437315], [5, 792, 448, 1663918149.437315], [6, 791, 445, 1663918149.437315], [7, 782, 430, 1663918149.437315], [8, 781, 424, 1663918149.437315], [9, 761, 462, 1663918149.437315], [10, 761, 458, 1663918149.437315], [11, 681, 409, 1663918149.437315], [12, 691, 414, 1663918149.437315], [13, 648, 526, 1663918149.437315], [14, 653, 543, 1663918149.437315], [15, 749, 538, 1663918149.437315], [16, 756, 541, 1663918149.437315], [17, 779, 539, 1663918149.437315], [18, 784, 541, 1663918149.437315], [19, 782, 526, 1663918149.437315], [20, 786, 524, 1663918149.437315], [21, 772, 522, 1663918149.437315], [22, 776, 522, 1663918149.437315], [23, 464, 439, 1663918149.437315], [24, 468, 453, 1663918149.437315], [25, 281, 477, 1663918149.437315], [26, 281, 489, 1663918149.437315], [27, 102, 469, 1663918149.437315], [28, 95, 475, 1663918149.4373

[[0, 774, 465, 1663918149.8831737], [1, 791, 454, 1663918149.8831737], [2, 792, 453, 1663918149.8831737], [3, 792, 451, 1663918149.8831737], [4, 791, 452, 1663918149.8831737], [5, 791, 449, 1663918149.8831737], [6, 792, 446, 1663918149.8831737], [7, 783, 428, 1663918149.8831737], [8, 781, 425, 1663918149.8831737], [9, 758, 462, 1663918149.8831737], [10, 757, 458, 1663918149.8831737], [11, 679, 406, 1663918149.8831737], [12, 691, 411, 1663918149.8831737], [13, 650, 522, 1663918149.8831737], [14, 655, 541, 1663918149.8831737], [15, 747, 530, 1663918149.8831737], [16, 757, 540, 1663918149.8831737], [17, 779, 534, 1663918149.8831737], [18, 783, 540, 1663918149.8831737], [19, 778, 519, 1663918149.8831737], [20, 781, 525, 1663918149.8831737], [21, 767, 514, 1663918149.8831737], [22, 772, 523, 1663918149.8831737], [23, 464, 442, 1663918149.8831737], [24, 469, 454, 1663918149.8831737], [25, 280, 479, 1663918149.8831737], [26, 281, 488, 1663918149.8831737], [27, 102, 468, 1663918149.8831737], [

[[0, 774, 466, 1663918150.343842], [1, 791, 454, 1663918150.343842], [2, 792, 453, 1663918150.343842], [3, 792, 451, 1663918150.343842], [4, 791, 453, 1663918150.343842], [5, 791, 450, 1663918150.343842], [6, 792, 447, 1663918150.343842], [7, 783, 428, 1663918150.343842], [8, 781, 425, 1663918150.343842], [9, 758, 462, 1663918150.343842], [10, 757, 459, 1663918150.343842], [11, 680, 406, 1663918150.343842], [12, 691, 411, 1663918150.343842], [13, 652, 522, 1663918150.343842], [14, 655, 541, 1663918150.343842], [15, 748, 530, 1663918150.343842], [16, 757, 540, 1663918150.343842], [17, 779, 534, 1663918150.343842], [18, 783, 540, 1663918150.343842], [19, 778, 519, 1663918150.343842], [20, 781, 526, 1663918150.343842], [21, 768, 515, 1663918150.343842], [22, 772, 523, 1663918150.343842], [23, 465, 442, 1663918150.343842], [24, 469, 454, 1663918150.343842], [25, 280, 479, 1663918150.343842], [26, 281, 487, 1663918150.343842], [27, 102, 468, 1663918150.343842], [28, 91, 472, 1663918150.3438

[[0, 774, 466, 1663918150.7927353], [1, 791, 455, 1663918150.7927353], [2, 792, 453, 1663918150.7927353], [3, 792, 451, 1663918150.7927353], [4, 791, 453, 1663918150.7927353], [5, 791, 450, 1663918150.7927353], [6, 792, 447, 1663918150.7927353], [7, 783, 428, 1663918150.7927353], [8, 781, 425, 1663918150.7927353], [9, 758, 462, 1663918150.7927353], [10, 757, 459, 1663918150.7927353], [11, 681, 406, 1663918150.7927353], [12, 691, 411, 1663918150.7927353], [13, 653, 522, 1663918150.7927353], [14, 656, 541, 1663918150.7927353], [15, 748, 530, 1663918150.7927353], [16, 758, 540, 1663918150.7927353], [17, 779, 534, 1663918150.7927353], [18, 783, 540, 1663918150.7927353], [19, 779, 520, 1663918150.7927353], [20, 781, 526, 1663918150.7927353], [21, 768, 515, 1663918150.7927353], [22, 772, 523, 1663918150.7927353], [23, 465, 442, 1663918150.7927353], [24, 469, 454, 1663918150.7927353], [25, 280, 478, 1663918150.7927353], [26, 281, 487, 1663918150.7927353], [27, 103, 468, 1663918150.7927353], [

[[0, 774, 466, 1663918151.0360744], [1, 791, 455, 1663918151.0360744], [2, 792, 453, 1663918151.0360744], [3, 792, 451, 1663918151.0360744], [4, 791, 453, 1663918151.0360744], [5, 791, 450, 1663918151.0360744], [6, 792, 447, 1663918151.0360744], [7, 783, 428, 1663918151.0360744], [8, 781, 425, 1663918151.0360744], [9, 758, 462, 1663918151.0360744], [10, 757, 459, 1663918151.0360744], [11, 681, 406, 1663918151.0360744], [12, 691, 411, 1663918151.0360744], [13, 653, 522, 1663918151.0360744], [14, 656, 541, 1663918151.0360744], [15, 748, 530, 1663918151.0360744], [16, 758, 540, 1663918151.0360744], [17, 779, 534, 1663918151.0360744], [18, 783, 540, 1663918151.0360744], [19, 779, 520, 1663918151.0360744], [20, 781, 526, 1663918151.0360744], [21, 768, 516, 1663918151.0360744], [22, 772, 523, 1663918151.0360744], [23, 465, 442, 1663918151.0360744], [24, 469, 455, 1663918151.0360744], [25, 280, 478, 1663918151.0360744], [26, 281, 487, 1663918151.0360744], [27, 103, 467, 1663918151.0360744], [

...

[[0, 773, 466, 1663918159.0972638], [1, 790, 454, 1663918159.0972638], [2, 791, 453, 1663918159.0972638], [3, 791, 451, 1663918159.0972638], [4, 790, 453, 1663918159.0972638], [5, 790, 450, 1663918159.0972638], [6, 790, 446, 1663918159.0972638], [7, 782, 428, 1663918159.0972638], [8, 780, 424, 1663918159.0972638], [9, 758, 463, 1663918159.0972638], [10, 757, 460, 1663918159.0972638], [11, 680, 407, 1663918159.0972638], [12, 690, 410, 1663918159.0972638], [13, 654, 524, 1663918159.0972638], [14, 656, 539, 1663918159.0972638], [15, 749, 532, 1663918159.0972638], [16, 761, 538, 1663918159.0972638], [17, 779, 536, 1663918159.0972638], [18, 784, 538, 1663918159.0972638], [19, 778, 521, 1663918159.0972638], [20, 783, 525, 1663918159.0972638], [21, 768, 516, 1663918159.0972638], [22, 774, 522, 1663918159.0972638], [23, 464, 442, 1663918159.0972638], [24, 469, 456, 1663918159.0972638], [25, 281, 476, 1663918159.0972638], [26, 282, 488, 1663918159.0972638], [27, 102, 466, 1663918159.0972638], [

[[0, 773, 466, 1663918159.5771148], [1, 790, 454, 1663918159.5771148], [2, 791, 453, 1663918159.5771148], [3, 791, 451, 1663918159.5771148], [4, 789, 453, 1663918159.5771148], [5, 790, 450, 1663918159.5771148], [6, 790, 446, 1663918159.5771148], [7, 782, 428, 1663918159.5771148], [8, 780, 424, 1663918159.5771148], [9, 758, 463, 1663918159.5771148], [10, 757, 460, 1663918159.5771148], [11, 679, 407, 1663918159.5771148], [12, 690, 410, 1663918159.5771148], [13, 654, 524, 1663918159.5771148], [14, 656, 539, 1663918159.5771148], [15, 749, 532, 1663918159.5771148], [16, 761, 538, 1663918159.5771148], [17, 779, 536, 1663918159.5771148], [18, 784, 538, 1663918159.5771148], [19, 778, 521, 1663918159.5771148], [20, 783, 525, 1663918159.5771148], [21, 768, 517, 1663918159.5771148], [22, 774, 522, 1663918159.5771148], [23, 464, 442, 1663918159.5771148], [24, 469, 456, 1663918159.5771148], [25, 281, 476, 1663918159.5771148], [26, 282, 488, 1663918159.5771148], [27, 102, 466, 1663918159.5771148], [

[[0, 773, 466, 1663918160.023943], [1, 790, 454, 1663918160.023943], [2, 790, 453, 1663918160.023943], [3, 791, 451, 1663918160.023943], [4, 789, 453, 1663918160.023943], [5, 790, 450, 1663918160.023943], [6, 790, 446, 1663918160.023943], [7, 782, 428, 1663918160.023943], [8, 780, 424, 1663918160.023943], [9, 757, 463, 1663918160.023943], [10, 756, 460, 1663918160.023943], [11, 679, 407, 1663918160.023943], [12, 690, 410, 1663918160.023943], [13, 654, 524, 1663918160.023943], [14, 656, 539, 1663918160.023943], [15, 749, 532, 1663918160.023943], [16, 761, 538, 1663918160.023943], [17, 779, 536, 1663918160.023943], [18, 784, 537, 1663918160.023943], [19, 778, 521, 1663918160.023943], [20, 783, 524, 1663918160.023943], [21, 768, 517, 1663918160.023943], [22, 773, 521, 1663918160.023943], [23, 464, 442, 1663918160.023943], [24, 469, 456, 1663918160.023943], [25, 281, 476, 1663918160.023943], [26, 282, 488, 1663918160.023943], [27, 101, 466, 1663918160.023943], [28, 89, 474, 1663918160.0239

[[0, 773, 466, 1663918160.489698], [1, 789, 454, 1663918160.489698], [2, 790, 453, 1663918160.489698], [3, 791, 451, 1663918160.489698], [4, 789, 453, 1663918160.489698], [5, 790, 450, 1663918160.489698], [6, 790, 446, 1663918160.489698], [7, 782, 428, 1663918160.489698], [8, 780, 424, 1663918160.489698], [9, 757, 463, 1663918160.489698], [10, 756, 460, 1663918160.489698], [11, 679, 407, 1663918160.489698], [12, 690, 410, 1663918160.489698], [13, 654, 524, 1663918160.489698], [14, 656, 539, 1663918160.489698], [15, 749, 532, 1663918160.489698], [16, 761, 538, 1663918160.489698], [17, 779, 536, 1663918160.489698], [18, 784, 537, 1663918160.489698], [19, 778, 521, 1663918160.489698], [20, 782, 524, 1663918160.489698], [21, 767, 517, 1663918160.489698], [22, 773, 521, 1663918160.489698], [23, 464, 442, 1663918160.489698], [24, 469, 456, 1663918160.489698], [25, 281, 476, 1663918160.489698], [26, 282, 488, 1663918160.489698], [27, 101, 466, 1663918160.489698], [28, 89, 474, 1663918160.4896

KeyboardInterrupt: 